# Mise en place d’un système de recommandation basé sur SVD

## Partie 1 : Préparation des données

#### 1. Chargement et exploration des données

In [15]:
import pandas as pd
import zipfile
import io
import requests
from sklearn.model_selection import train_test_split

# a. Téléchargez les données MovieLens disponibles à l’URL suivante :
url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# b. Chargez le fichier ratings.csv dans un DataFrame pandas.
with zip_file.open("ml-latest-small/ratings.csv") as file:
    ratings_df = pd.read_csv(file)

# c. Affichez les cinq premières lignes du DataFrame.
print(ratings_df.head())

# d. Décrivez brièvement les colonnes suivantes :
# - userId : Identifiant unique de l'utilisateur
# - movieId : Identifiant unique du film
# - rating : Note donnée par l'utilisateur au film

# e. Divisez les données en trois ensembles : training data (60 %), validation data (20 %) et test data (20 %).
train_df, temp_df = train_test_split(ratings_df, test_size=0.4, random_state=42)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Vérification des tailles des ensembles
print(f"Training data size: {len(train_df)}")
print(f"Validation data size: {len(validation_df)}")
print(f"Test data size: {len(test_df)}")



   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
Training data size: 60501
Validation data size: 20167
Test data size: 20168


#### 2. Création de la matrice utilisateur-film

In [20]:
# a. Transformez le DataFrame en une matrice utilisateur-film en utilisant la méthode pivot de pandas :
user_movie_matrix = ratings_df.pivot(index='userId', columns='movieId', values='rating')

# b. Remplacez les valeurs manquantes dans la matrice par 0.
user_movie_matrix = user_movie_matrix.fillna(0)

# c. Affichez la matrice utilisateur-film obtenue.
print(user_movie_matrix)


movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     0.0     4.0     0.0     0.0     4.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
606         2.5     0.0     0.0     0.0     0.0     0.0     2.5     0.0   
607         4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
608         2.5     2.0     2.0     0.0     0.0     0.0     0.0     0.0   
609         3.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
610         5.0     0.0     0.0     0.0     0.0     5.0     0.0     0.0   

movieId  9       10     

## Partie 2 : Manipulation des matrices

#### 3. Conversion en matrice sparse

In [25]:
import scipy.sparse as sp

# a. Convertissez la matrice utilisateur-film en une matrice sparse en utilisant la classe csr_matrix de la bibliothèque scipy.
user_movie_matrix_sparse = sp.csr_matrix(user_movie_matrix.values)

Pourquoi utiliser une matrice sparse dans ce contexte ?

Dans le contexte des recommandations de films, la matrice utilisateur-film est généralement très creuse, c'est-à-dire qu'elle contient beaucoup de zéros. Utiliser une matrice sparse permet de stocker et de manipuler cette matrice de manière plus efficace en termes de mémoire et de performance. Les matrices sparse ne stockent que les éléments non nuls, ce qui réduit considérablement la consommation de mémoire et accélère les opérations mathématiques sur la matrice.

#### 4. Décomposition SVD

In [29]:
from scipy.sparse.linalg import svds

# a. Effectuez une décomposition en valeurs singulières (SVD) sur la matrice sparse avec k = 20.
k = 20
U, sigma, VT = svds(user_movie_matrix_sparse, k=k)

# b. Affichez les dimensions des matrices résultantes : U, σ, et V^T.
print(f"Dimensions de U: {U.shape}")
print(f"Dimensions de σ: {sigma.shape}")
print(f"Dimensions de V^T: {VT.shape}")

Dimensions de U: (610, 20)
Dimensions de σ: (20,)
Dimensions de V^T: (20, 9724)


Explication

Rôle de chaque matrice dans le cadre d’un système de recommandation :

- **U** : La matrice \( U \) contient les vecteurs latents des utilisateurs. Chaque ligne de \( U \) représente un utilisateur dans un espace de dimension réduite (k dimensions). Ces vecteurs capturent les préférences des utilisateurs. En d'autres termes, chaque utilisateur est représenté par un vecteur de k dimensions qui résume ses goûts et ses préférences.

- **σ** : La matrice \( \sigma \) (ou sigma) est une matrice diagonale contenant les valeurs singulières. Ces valeurs indiquent l'importance de chaque dimension latente. Plus une valeur singulière est grande, plus la dimension correspondante est importante pour expliquer les données. Les valeurs singulières permettent de pondérer l'importance des dimensions latentes dans la reconstruction de la matrice utilisateur-film.

- **V^T** : La matrice \( V^T \) contient les vecteurs latents des films. Chaque colonne de \( V^T \) représente un film dans un espace de dimension réduite (k dimensions). Ces vecteurs capturent les caractéristiques des films. En d'autres termes, chaque film est représenté par un vecteur de k dimensions qui résume ses caractéristiques et ses attributs.

En utilisant ces matrices, un système de recommandation peut prédire les notes que les utilisateurs donneraient à des films qu'ils n'ont pas encore vus, en projetant les utilisateurs et les films dans un espace latent commun et en calculant les similarités entre eux.


#### 5. Reconstruction de la matrice

In [34]:
import numpy as np
import pandas as pd

# a. Transformez le vecteur σ en une matrice diagonale.
sigma_matrix = np.diag(sigma)

# b. Reconstituez la matrice utilisateur-film approximée en utilisant la formule :
# predicted_ratings = U · σ · V^T
predicted_ratings = np.dot(np.dot(U, sigma_matrix), VT)

# c. Affichez la matrice reconstituée sous forme de DataFrame pandas.
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=user_movie_matrix.index, columns=user_movie_matrix.columns)
print(predicted_ratings_df)


movieId    1         2         3         4         5         6         7       \
userId                                                                          
1        2.290336  1.460203  1.033507 -0.061334 -0.002275  1.243261  0.029650   
2        0.038570  0.015272  0.016968  0.002944  0.019201 -0.005821 -0.025436   
3       -0.015220  0.049067  0.047202 -0.004936 -0.035349  0.052758 -0.012911   
4        2.238621  0.060011  0.039384  0.066455  0.221806  0.487591  0.318594   
5        1.358363  0.970071  0.340939  0.121053  0.479936  0.628346  0.504583   
...           ...       ...       ...       ...       ...       ...       ...   
606     -0.617336  0.556016 -0.374855  0.162583 -0.155438 -1.403045  2.364098   
607      2.056401  1.216670  0.593186 -0.006625 -0.020369  1.678307  0.261799   
608      2.369716  1.838958  1.577564 -0.131902  0.362084  3.628608  0.248347   
609      0.809741  0.651456  0.297184  0.081167  0.334388  0.577311  0.362697   
610      5.175515  0.409812 

## Partie 3 : Système de recommandation

#### 6. Implémentation d’une fonction de recommandation

In [40]:
def recommend_movies(user_id, predicted_ratings_df, ratings_df, num_recommendations=5):
    """
    Recommande des films à un utilisateur en fonction des prédictions de la matrice utilisateur-film.

    Paramètres :
    - user_id : l’ID de l’utilisateur pour lequel recommander des films.
    - predicted_ratings_df : la matrice utilisateur-film prédite.
    - ratings_df : le DataFrame original contenant les notes.
    - num_recommendations (optionnel) : le nombre de recommandations à retourner (par défaut : 5).

    Retourne :
    - Une liste de tuples contenant les ID des films recommandés et leurs notes prédites.
    """
    # Identifier les films déjà notés par l’utilisateur
    rated_movies = ratings_df[ratings_df['userId'] == user_id]['movieId']

    # Trier les prédictions des films non notés dans l’ordre décroissant
    user_predictions = predicted_ratings_df.loc[user_id].drop(rated_movies)
    sorted_predictions = user_predictions.sort_values(ascending=False)

    # Retourner les num_recommendations meilleures recommandations
    top_recommendations = sorted_predictions.head(num_recommendations)

    return top_recommendations.to_dict()

# Exemple d'utilisation de la fonction
user_id = 1  # Remplacez par l'ID de l'utilisateur pour lequel vous voulez des recommandations
recommendations = recommend_movies(user_id, predicted_ratings_df, ratings_df)
print(recommendations)


{589: 3.931090028958753, 1200: 3.725028597671198, 2762: 3.334133498541925, 1968: 3.317303918598695, 1036: 3.160154774045102}


#### 7. Test du système de recommandation

In [43]:
# a. Testez la fonction de recommandation pour un utilisateur aléatoire ou spécifique (par exemple, user_id = 1).
user_id = 1  # Remplacez par l'ID de l'utilisateur pour lequel vous voulez des recommandations
recommendations = recommend_movies(user_id, predicted_ratings_df, ratings_df)

# b. Affichez les films recommandés et expliquez les résultats obtenus.
print(f"Recommandations pour l'utilisateur {user_id}:")
for movie_id, predicted_rating in recommendations.items():
    print(f"Film ID: {movie_id}, Note prédite: {predicted_rating:.2f}")

Recommandations pour l'utilisateur 1:
Film ID: 589, Note prédite: 3.93
Film ID: 1200, Note prédite: 3.73
Film ID: 2762, Note prédite: 3.33
Film ID: 1968, Note prédite: 3.32
Film ID: 1036, Note prédite: 3.16


Explication des résultats obtenus

Recommandations pour l'utilisateur

La fonction `recommend_movies` retourne une liste de films recommandés pour l'utilisateur spécifié par `user_id`. Chaque recommandation est accompagnée d'une note prédite, qui est une estimation de la note que l'utilisateur donnerait à ce film.

Films recommandés

Les films recommandés sont ceux qui ont les notes prédites les plus élevées parmi les films que l'utilisateur n'a pas encore notés. Cela signifie que ces films sont susceptibles de plaire à l'utilisateur en se basant sur les préférences latentes capturées par la décomposition SVD.

Utilisation de la matrice utilisateur-film prédite

La fonction utilise la matrice utilisateur-film prédite pour identifier les films qui sont susceptibles de plaire à l'utilisateur. Cette matrice contient des prédictions de notes pour tous les utilisateurs et tous les films, même ceux qui n'ont pas encore été notés par l'utilisateur. En se basant sur ces prédictions, la fonction peut recommander des films qui correspondent aux préférences de l'utilisateur.

Préférences latentes

Les préférences latentes capturées par la décomposition SVD permettent de représenter les utilisateurs et les films dans un espace de dimension réduite. Cela permet de calculer des similarités entre les utilisateurs et les films, et de faire des recommandations basées sur ces similarités.

En résumé, la fonction de recommandation utilise les prédictions de la matrice utilisateur-film pour identifier les films qui sont susceptibles de plaire à l'utilisateur, en se basant sur les préférences latentes capturées par la décomposition SVD. Les films recommandés sont ceux qui ont les notes prédites les plus élevées parmi les films que l'utilisateur n'a pas encore notés.


In [47]:
import numpy as np
from sklearn.metrics import mean_squared_error

def calculate_rmse(predicted_ratings_df, test_df):
    """
    Calcule l'erreur quadratique moyenne (RMSE) entre les prédictions et les notes réelles.

    Paramètres :
    - predicted_ratings_df : la matrice utilisateur-film prédite.
    - test_df : le DataFrame contenant les notes réelles de l'ensemble de test.

    Retourne :
    - La valeur RMSE.
    """
    # Filtrez les données pour ne comparer que les films réellement notés dans l’ensemble de test
    test_user_ids = test_df['userId'].values
    test_movie_ids = test_df['movieId'].values
    test_ratings = test_df['rating'].values

    predicted_ratings = []
    for user_id, movie_id in zip(test_user_ids, test_movie_ids):
        predicted_rating = predicted_ratings_df.loc[user_id, movie_id]
        predicted_ratings.append(predicted_rating)

    # Calculez la RMSE
    rmse = np.sqrt(mean_squared_error(test_ratings, predicted_ratings))
    return rmse

# Calculez et affichez la valeur RMSE du modèle
rmse = calculate_rmse(predicted_ratings_df, test_df)
print(f"RMSE du modèle : {rmse:.2f}")


RMSE du modèle : 2.37


# Analyse des résultats

## Interprétation de la valeur de RMSE

La valeur de RMSE (Root Mean Squared Error) obtenue nous donne une idée de la qualité des prédictions du modèle. Plus la valeur de RMSE est faible, meilleures sont les prédictions. Une RMSE élevée indique que les prédictions sont loin des valeurs réelles, ce qui signifie que le modèle a du mal à capturer les préférences des utilisateurs.

## Pistes pour améliorer les performances du modèle

1. **Augmenter la dimension latente (k)** : Utiliser une valeur plus élevée pour \( k \) dans la décomposition SVD peut permettre de capturer plus de détails dans les préférences des utilisateurs et les caractéristiques des films.

2. **Utiliser des techniques de régularisation** : Ajouter une régularisation à la décomposition SVD peut aider à éviter le surapprentissage et améliorer la généralisation du modèle.

3. **Incorporer des informations supplémentaires** : Intégrer des informations supplémentaires sur les utilisateurs et les films (comme les genres de films, les profils des utilisateurs, etc.) peut améliorer la précision des prédictions.

4. **Utiliser des modèles plus avancés** : Explorer des modèles plus avancés comme les réseaux de neurones ou les modèles de factorisation matricielle avec des contraintes supplémentaires peut améliorer les performances.

5. **Traitement des valeurs aberrantes** : Identifier et traiter les valeurs aberrantes dans les données peut améliorer la qualité des prédictions.

6. **Validation croisée** : Utiliser des techniques de validation croisée pour mieux évaluer les performances du modèle et ajuster les hyperparamètres en conséquence.
